# Tensorflow Recommendation: Basics

In [1]:
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

In [2]:
# movie lens data

ratings = tfds.load("movie_lens/100k-ratings", split='train')

movies = tfds.load("movie_lens/100k-ratings", split='train')

In [3]:
ratings = ratings.map(lambda x: {
    'movie_title': x['movie_title'],
    'user_id': x['user_id'],
})

movies = movies.map(lambda x: x['movie_title'])

In [4]:
# build vocab to convert user ids and movie titles to integer indices for embedding layers
user_id_vocab = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
user_id_vocab.adapt(ratings.map(lambda x: x['user_id']))

movie_title_vocab = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
movie_title_vocab.adapt(movies)

In [13]:
from typing import Dict, Text

In [19]:
# define model

class MovieLensModel(tfrs.Model):
  # We derive from a custom base class to help reduce boilerplate. Under the hood,
  # these are still plain Keras Models.

    def __init__(
      self,
      user_model: tf.keras.Model,
      movie_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
        super().__init__()

    # Set up user and movie representations.
        self.user_model = user_model
        self.movie_model = movie_model

    # Set up a retrieval task.
        self.task = task

    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.

        user_embeddings = self.user_model(features["user_id"])
        movie_embeddings = self.movie_model(features["movie_title"])

        return self.task(user_embeddings, movie_embeddings)

    # Define user and movie models.
    user_model = tf.keras.Sequential([
    user_id_vocab,
    tf.keras.layers.Embedding(user_id_vocab.vocab_size(), 64)
])
    movie_model = tf.keras.Sequential([
    movie_title_vocab,
    tf.keras.layers.Embedding(movie_title_vocab.vocab_size(), 64)
])

# Define your objectives.
    task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    movies.batch(128).map(movie_model)
  )
)
    
    # create a Retrieval model
    model = MovieLensModel(user_model, movie_model, task)
    model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))


    model.fit(ratings.batch(4096), epochs=5)

# use brute force for retrieval
    index = tfrs.layers.ann.BruteForce(model.user_model)
    index.index(movies.batch(100).map(model.movie_model), movies)

    _, titles = index(np.array(["42"]))
    print(f"Top 3 recommendations for user 42: {titles[0, :3]}")

Epoch 1/5
25/25 [==============================] - 275s 11s/step - loss: 32418.9278 - regularization_loss: 0.0000e+00 - total_loss: 32418.9278
Epoch 2/5
25/25 [==============================] - 271s 11s/step - loss: 32418.9278 - regularization_loss: 0.0000e+00 - total_loss: 32418.9278
Epoch 3/5
25/25 [==============================] - 275s 11s/step - loss: 32418.9278 - regularization_loss: 0.0000e+00 - total_loss: 32418.9278
Epoch 4/5
25/25 [==============================] - 274s 11s/step - loss: 32418.9278 - regularization_loss: 0.0000e+00 - total_loss: 32418.9278
Epoch 5/5
25/25 [==============================] - 273s 11s/step - loss: 32418.9278 - regularization_loss: 0.0000e+00 - total_loss: 32418.9278


NameError: name 'np' is not defined